In [ ]:
import numpy as np
import scipy.io
import os
import pickle
import matplotlib.pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay

# Download and prepare Dataset

In [ ]:
candidate_paths = [
    './datasets',
]

# constants for each dataset
csi_per_second   =   150
first_subcarrier = -1024
last_subcarrier  =  1023

# csi per second after downsampling
new_csi_per_second = 30

# seconds in file
seconds = 80

# considered time window in seconds
time_window = 3


actions = [('A','Walk'), ('B', 'Run'), ('C', 'Jump'), ('D', 'Sitting'), 
           ('E', 'Empty Room'), ('F', 'Stand'),('G', 'Wave hands'),  ('H', 'Clapping'),
           ('I', 'Lay'), ('J', 'Wiping'), ('K', 'Squat'), ('L', 'Stretching')] 
downsampling_modes = ['mean', 'median', 'min', 'max']

In [ ]:
!wget https://zenodo.org/records/7751897/files/S1.zip
!unzip S1.zip
!mv -v S1/* datasets/dataset/
!rm S1.zip
!rm -r S1

--2024-07-04 12:41:45--  https://zenodo.org/records/7751897/files/S4.zip
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11019742449 (10G) [application/octet-stream]
Saving to: ‘S4.zip’

S4.zip                3%[                    ] 324.50M  50.0MB/s    eta 3m 57s 

S4.zip              100%[===================>]  10.26G  48.0MB/s    in 3m 41s  

2024-07-04 12:45:25 (47.7 MB/s) - ‘S4.zip’ saved [11019742449/11019742449]

Archive:  S4.zip
   creating: S4/
  inflating: S4/LICENSE              
  inflating: S4/README               
  inflating: S4/S4a_A.mat            
  inflating: S4/S4a_B.mat            
  inflating: S4/S4a_C.mat            
  inflating: S4/S4a_D.mat            
  inflating: S4/S4a_E.mat            
  inflating: S4/S4a_F.mat            
  inflating: S4/S4a_G.mat            
  inflating: S4/S4a_H.mat            
  inflating: S4/S4a_I.mat            
  inflating: S4/S4a_J.mat            
  inflating: S4/S4a_K.mat            
  inflating: S4/S4a_L.mat            
  inflating: S4/S4b_A.mat            
  inflating: S4/S4b_B.mat            
  inflating: S4/S4b_C.mat            
  inflating: S4/S4b_D.mat            
  inflating: S4/S4b_E.mat            
  inflating: S4/S4b_F.mat            
  inflating: S4/S4b_G.mat            
  inflating

In [ ]:
# create files with downsampled csi
def downsample(candidate_paths, csi_per_second, new_csi_per_second, actions):
    for action in actions:
        filename = f'S4a_{action[0]}'
        # Read CSI from the first file found among the candidates
        for path in candidate_paths:
          full_path_name = f'{path}/dataset/{filename}.mat'
          if os.path.exists(full_path_name):
            csi = scipy.io.loadmat(full_path_name)['csi']
          else:
            print(f'File {full_path_name} not found')
        
        csi = np.abs(csi)
        data_reshaped = csi.reshape(-1, int(csi_per_second/new_csi_per_second), 2048, 4) #[2400, 5, 2048, 4]

        
        # Aggregation functions
        csi_mean = data_reshaped.mean(axis=1) #[2400,2048,4]
        csi_median = np.median(data_reshaped, axis=1)
        csi_max = data_reshaped.max(axis=1)
        csi_min = data_reshaped.min(axis=1)

        # Save the new CSI data
        with open(f'{path}/mean_dataset_abs/{filename}.pkl', 'wb') as f:
            pickle.dump(csi_mean, f)
            print(f'saved mean {action[1]}')

        with open(f'{path}/median_dataset_abs/{filename}.pkl', 'wb') as f:
            pickle.dump(csi_median, f)
            print(f'saved median {action[1]}')

        with open(f'{path}/max_dataset_abs/{filename}.pkl', 'wb') as f:
            pickle.dump(csi_max, f)
            print(f'saved max {action[1]}')
        
        with open(f'{path}/min_dataset_abs/{filename}.pkl', 'wb') as f:
            pickle.dump(csi_min, f)
            print(f'saved min {action[1]}')

downsample(candidate_paths, csi_per_second, new_csi_per_second, actions)

saved mean Walk
saved median Walk
saved max Walk
saved min Walk


saved mean Run
saved median Run
saved max Run
saved min Run
saved mean Jump
saved median Jump
saved max Jump
saved min Jump
saved mean Sitting
saved median Sitting
saved max Sitting
saved min Sitting
saved mean Empty Room
saved median Empty Room
saved max Empty Room
saved min Empty Room
saved mean Stand
saved median Stand
saved max Stand
saved min Stand
saved mean Wave hands
saved median Wave hands
saved max Wave hands
saved min Wave hands
saved mean Clapping
saved median Clapping
saved max Clapping
saved min Clapping
saved mean Lay
saved median Lay
saved max Lay
saved min Lay
saved mean Wiping
saved median Wiping
saved max Wiping
saved min Wiping
saved mean Squat
saved median Squat
saved max Squat
saved min Squat
saved mean Stretching
saved median Stretching
saved max Stretching
saved min Stretching


# Baseline SNN vs CNN

In [ ]:
LABELS = ['Walk', 'Run', 'Jump', 'Wave hands', 'Clapping', 'Wiping', 'Squat']
from SNN.neural import main as train
from SNN.metrics import bayesianHypothesisTesting

In [ ]:
cmSNN, cmCNN = train()

# Print the confusion matrix
print("Confusion Matrix:")
disp = ConfusionMatrixDisplay(confusion_matrix=cmSNN, display_labels=LABELS)
cmdisp = disp.plot(cmap="cividis")
plt.setp(cmdisp.ax_.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.setp(cmdisp.ax_.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
cmdisp.figure_.set_size_inches(12, 12)
cmdisp.figure_.savefig("SNN/results/neuralOnly/ConMatSNN1s.png", bbox_inches='tight')


# Print the confusion matrix
print("Confusion Matrix:")
disp = ConfusionMatrixDisplay(confusion_matrix=cmCNN, display_labels=LABELS)
cmdisp = disp.plot(cmap="cividis")
plt.setp(cmdisp.ax_.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.setp(cmdisp.ax_.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
cmdisp.figure_.set_size_inches(12, 12)
cmdisp.figure_.savefig("SNN/results/neuralOnly/ConfMatCnn1s.png", bbox_inches='tight')

print(f'Accuracy SNN: {np.trace(cmSNN)/np.sum(cmSNN)}')
print(f'Accuracy CNN: {np.trace(cmCNN)/np.sum(cmCNN)}')

bayesianHypothesisTesting(cmSNN, cmCNN)

# Neurosymbolic SNN

In [ ]:
from SNN.results import main as neuroSpykeHar
from SNN.results import prepare_data
from SNN.deepprobhar import main as deepProbHar
from SNN.metrics import bayesianHypothesisTesting


In [ ]:
train, test, loader = prepare_data()

cmSNN = neuroSpykeHar()
cmVAE = deepProbHar()


# Save the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cmSNN, display_labels=LABELS)
cmdisp = disp.plot(cmap="cividis")
plt.setp(cmdisp.ax_.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.setp(cmdisp.ax_.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
cmdisp.figure_.set_size_inches(12, 12)
cmdisp.figure_.savefig(f"SNN/results/neuroSymbolic/ConfMatSNN.png", bbox_inches='tight')

	# Save the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cmVAE, display_labels=LABELS)
cmdisp = disp.plot(cmap="cividis")
plt.setp(cmdisp.ax_.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.setp(cmdisp.ax_.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
cmdisp.figure_.set_size_inches(12, 12)
cmdisp.figure_.savefig(f"SNN/results/neuroSymbolic/ConfMatVAE.png", bbox_inches='tight')

print(f'Accuracy SNN: {np.trace(cmSNN)/np.sum(cmSNN)}')
print(f'Accuracy CNN: {np.trace(cmVAE)/np.sum(cmVAE)}')

bayesianHypothesisTesting(cmSNN, cmVAE)

# Temporal Neurosymbolic SNN

In [ ]:
from SNN.nsTrain import main as nsTrain

nsTrain()